## Group 33

- Alexandre Gonçalves (20240738)
- Bràulio Damba (20240007)
- João Henriques (20240499)
- Mariana Sousa (20240516)
- Victoria Goon (20240550)

In [1]:
import numpy as np
from pathlib import Path
from Library.utils import get_callbacks
from sklearn.metrics import classification_report

import tensorflow as tf

from keras.utils import image_dataset_from_directory
from keras.applications import VGG16, InceptionV3, ResNet50
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, F1Score, Precision, Recall

from Library.PreprocessingAugmentTemplate import Preprocessing_Augment_Template

In [2]:
n_classes = 202

In [3]:
# CHANGE ONLY HERE
model_name = "VGG16"
epochs = 64
resolution = 224
base_model = VGG16(include_top=False, classes=n_classes, input_shape=(resolution,resolution,3), weights="imagenet")
trainable = False
batch_size = 64
augmentation = False

In [4]:
train_data = image_dataset_from_directory(
    Path("data/train"),
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(resolution,resolution)
)

Found 7622 files belonging to 202 classes.


In [5]:
val_data = image_dataset_from_directory(
    Path("data/val"),
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(resolution,resolution)
)

Found 1558 files belonging to 202 classes.


In [6]:
test_data = image_dataset_from_directory(
    Path("data/test"),
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(resolution,resolution)
)

Found 1798 files belonging to 202 classes.


In [ ]:
model = Preprocessing_Augment_Template(
    base_model=base_model,
    augment=augmentation,
    resolution=resolution,
    trainable = trainable
)
model.build(input_shape=(None, resolution, resolution, 3))

optimizer = Adam(learning_rate=0.001, name="optimizer")
loss = CategoricalCrossentropy(name="loss")

AttributeError: 'KerasTensor' object has no attribute 'build'

In [ ]:
# metrics
categorical_accuracy = CategoricalAccuracy(name="accuracy")
f1_score = F1Score(average="macro", name="f1_score")
metrics = [categorical_accuracy, f1_score, Precision(), Recall()]

# callbacks
callbacks = get_callbacks(
    base_dir="model_logs/" + model_name
)

In [ ]:
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
# print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
_ = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=callbacks,
    verbose=2
)

In [ ]:
# evaluate on the test ds
model.evaluate(
    test_data,
    return_dict=True,
    verbose=1
)

In [ ]:
y_true = []
y_pred = []
class_names = train_data.class_names
 
for images, labels in test_data:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))
 
print(classification_report(y_true, y_pred, target_names=class_names, digits=3))